In [ ]:
%%writefile Proyecto_Final.py

######################################
#Importamos librerias
import streamlit as st
import plotly.express as px
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score
import re



######################################

st.markdown("""
    <style>
    /* Fondo del cuerpo */
    .stApp {
        background-color: #f0f0f0;
        color: black;
    }

    /* Sidebar oscuro */
    section[data-testid="stSidebar"] {
        background-color: #111;
    }

    /* Títulos y etiquetas del sidebar en blanco */
    section[data-testid="stSidebar"] label,
    section[data-testid="stSidebar"] h1,
    section[data-testid="stSidebar"] h2,
    section[data-testid="stSidebar"] h3,
    section[data-testid="stSidebar"] h4 {
        color: white !important;
        font-weight: bold;
    }

    /* Inputs del sidebar con texto negro y fondo blanco */
    section[data-testid="stSidebar"] input,
    section[data-testid="stSidebar"] textarea,
    section[data-testid="stSidebar"] select {
        color: black !important;
        background-color: white !important;
    }

    /* Header superior transparente */
    header[data-testid="stHeader"] {
        background-color: transparent;
        border: none;
    }

    /* Títulos generales del cuerpo */
    h1, h2, h3, h4 {
        color: black;
    }

    /* Espaciado general */
    .block-container {
        padding-top: 2rem;
        padding-bottom: 2rem;
    }
    </style>
""", unsafe_allow_html=True)



@st.cache_resource
def load_data():
    def clean_price_column(df):
        df['price'] = df['price'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
        df['price'] = pd.to_numeric(df['price'], errors='coerce')
        df = df.dropna(subset=['price']) 
        return df

    def clean_binary_columns(df):
        binary_columns = ['host_identity_verified', 'has_availability', 'instant_bookable', 'host_is_superhost']
        for col in binary_columns:
            df[col] = df[col].fillna('Sin identificar')
            df[col] = df[col].map({'t': 1, 'f': 0}).fillna(0).astype(int)
        return df

    def add_engineered_features(df):
        df["bathroomsd"] = df["bathrooms_text"].astype(str).apply(lambda x: 1 if "2" in x or "3" in x or "4" in x else 0)
        df["E_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Entire home/apt" else 0)
        df["P_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Private room" else 0)
        df["H_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Hotel room" else 0)
        df["instant_bookabled"] = df["instant_bookable"].apply(lambda x: 1 if x == "t" else 0)
        df["price"] = df["price"].replace('[\$,]', '', regex=True).astype(float)
        df["D_price"] = df["price"].apply(lambda x: 1 if x >= df["price"].median() else 0)
        df["D_host_listings"] = df["host_total_listings_count"].apply(lambda x: 1 if x >= df["host_total_listings_count"].median() else 0)
        df["D_accuracy"] = df["review_scores_accuracy"].apply(lambda x: 1 if x >= df["review_scores_accuracy"].median() else 0)
        df["D_availability"] = df["availability_30"].apply(lambda x: 1 if x >= df["availability_30"].median() else 0)
        df["D_availability90"] = df["availability_90"].apply(lambda x: 1 if x >= df["availability_90"].median() else 0)
        df["D_reviews"] = df["number_of_reviews"].apply(lambda x: 1 if x >= df["number_of_reviews"].median() else 0)
        df["D_rating"] = df["review_scores_rating"].apply(lambda x: 1 if x >= df["review_scores_rating"].median() else 0)
        df["D_accommodates"] = df["accommodates"].apply(lambda x: 1 if x >= df["accommodates"].median() else 0)
        df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0, 'Sin identificar': 0}).fillna(0).astype(int)
        return df

    #Roma
    df = pd.read_csv('Roma.csv')
    df = clean_price_column(df)
    df1 = pd.read_csv('Roma1.csv')
    df1 = clean_price_column(df1)
    df1 = clean_binary_columns(df1)
    df2 = pd.read_csv('Roma.csv')
    df2 = add_engineered_features(df2)

    #Mexico
    dfM = pd.read_csv('Mexico.csv')
    dfM = clean_price_column(dfM)
    df1M = pd.read_csv('Mexico1.csv')
    df1M = clean_price_column(df1M)
    df1M = clean_binary_columns(df1M)
    df2M = pd.read_csv('Mexico.csv')
    df2M = add_engineered_features(df2M)

    #Venecia
    dfV = pd.read_csv('Venecia.csv')
    dfV = clean_price_column(dfV)
    df1V = pd.read_csv('Venecia1.csv')
    df1V = clean_price_column(df1V)
    df1V = clean_binary_columns(df1V)
    df2V = pd.read_csv('Venecia.csv')
    df2V = add_engineered_features(df2V)
    
    #Selecciono las columnas tipo numericas del dataframe
    numeric_df = df.select_dtypes(['float','int']) 
    numeric_cols = numeric_df.columns            

    #Selecciono las columnas tipo texto del dataframe
    text_df = df.select_dtypes(['object'])
    text_cols = text_df.columns

    #Obtengo el numero de filas y el numero de columnas del dataset
    num_filas = df.shape[0]
    num_columnas = df.shape[1]

    #Selecciono algunas columnas categoricas de valores para desplegar en diferentes widgets
    categorical_column_room_type = df['room_type']

    #Obtengo los valores unicos de la columna categorica seleccionada
    categorical_column_room_type = categorical_column_room_type.unique()

    return df,df1,df2,dfM,df1M,df2M,dfV,df1V,df2V, numeric_cols, text_cols, categorical_column_room_type, numeric_df, num_filas, num_columnas

df, df1,df2,dfM,df1M,df2M,dfV,df1V,df2V, numeric_cols, text_cols, categorical_column_room_type, numeric_df, num_filas, num_columnas = load_data()

######################################################################
#CREACION DE DASHBOARD
#Generamos las paginas que utilizaremos en el diseño
#witget 1 = Selectbox
#Menu desplegable de opciones de las paginas seleccionadas
View = st.selectbox(label='View', options=["Introduccion", "Extraccion de caracteristicas" , "Regresion Lineal Simple", "Regresion Lineal Multiple", "Regresion Logistica"])


#######################################################################################################################################
#Contenido de la vista 1
if View == 'Introduccion':

    st.title("Bienvenido al Dashboard")
    st.markdown("Este panel analiza datos de alojamiento de tres ciudades.")
    st.markdown("### Vistas disponibles:")
    st.markdown("- Introducción")
    st.markdown("- Extracción de características")
    st.markdown("- Regresión Lineal Simple")
    st.markdown("- Regresión Lineal Múltiple")
    st.markdown("- Regresión Logística")

    st.sidebar.markdown("### Mostrar Dataset por ciudad:")
    mostrar_roma = st.sidebar.button("Roma 🇮🇹")
    mostrar_mexico = st.sidebar.button("Ciudad de México 🇲🇽")
    mostrar_venecia = st.sidebar.button("Venecia 🇮🇹")

    st.header("Información general del dataset principal (Roma)")
    st.write("Número total de filas:", num_filas)
    st.write("Número total de columnas:", num_columnas)

    if mostrar_roma:
        st.subheader("Dataset - Roma 🇮🇹")
        st.image("broma.png", width=100)
        st.dataframe(df)

    if mostrar_mexico:
        st.subheader("Dataset - Ciudad de México 🇲🇽")
        st.image("bmexico.jpg", width=100)
        st.dataframe(dfM)

    if mostrar_venecia:
        st.subheader("Dataset - Venecia 🇮🇹")
        st.image("bvenecia.jpg", width=100)
        st.dataframe(dfV)

#######################################################################################################################################

#Viwe 2
#####################################################################
# Vista 2 - Extracción de características
# Vista 2 - Extracción de características
if View == 'Extraccion de caracteristicas':
    st.title('Extracción de características')

    # Sidebar - Controles
    st.sidebar.header("Opciones de visualización")

    chart_type = st.sidebar.selectbox(
        "Tipo de gráfico para variables categóricas:",
        ["Gráfico Circular", "Gráfico de Barras", "Tabla de Frecuencias"]
    )

    # Determinar columnas categóricas comunes entre datasets
    common_categorical = list(
        set(df.select_dtypes(['object']).columns)
        & set(dfM.select_dtypes(['object']).columns)
        & set(dfV.select_dtypes(['object']).columns)
    )

    selected_column = st.sidebar.selectbox(
        "Columna categórica a visualizar:",
        common_categorical
    )

    # --- Mostrar distribución categórica por ciudad ---
    st.header("Distribución categórica por ciudad")

    def mostrar_ciudad_con_grafico(df_ciudad, nombre_ciudad, bandera_path, color):
        st.subheader(nombre_ciudad)
        st.image(bandera_path, width=100)

        counts = df_ciudad[selected_column].value_counts().reset_index()
        counts.columns = [selected_column, 'count']

        if chart_type == "Tabla de Frecuencias":
            st.dataframe(counts)
        elif chart_type == "Gráfico Circular":
            fig = px.pie(
                data_frame=counts,
                names=selected_column,
                values='count',
                title=f'{selected_column} en {nombre_ciudad}',
                color_discrete_sequence=[color],
                width=600,
                height=400
            )
            st.plotly_chart(fig)
        elif chart_type == "Gráfico de Barras":
            fig_bar = px.bar(
                data_frame=counts,
                x=selected_column,
                y='count',
                title=f'{selected_column} en {nombre_ciudad}',
                color_discrete_sequence=[color],
                width=600,
                height=400
            )
            st.plotly_chart(fig_bar)

    mostrar_ciudad_con_grafico(df, "Roma", "broma.png", "blue")
    mostrar_ciudad_con_grafico(dfM, "Ciudad de México", "bmexico.jpg", "green")
    mostrar_ciudad_con_grafico(dfV, "Venecia", "bvenecia.jpg", "red")

    # --- Filtro por precio ---
    st.header("Filtrado por precio por ciudad")

    def mostrar_precio_ciudad(df_ciudad, nombre_ciudad):
        st.subheader(nombre_ciudad)
        max_price = int(df_ciudad['price'].max())
        price_limit = st.slider(f"{nombre_ciudad} - Precio máximo:", 0, max_price, max_price // 2, step=1, key=nombre_ciudad)

        filtered = df_ciudad[df_ciudad['price'] <= price_limit]
        st.write(f"{len(filtered)} alojamientos con precio ≤ {price_limit}")
        cols = filtered.columns.tolist()
        cols.insert(0, cols.pop(cols.index('price')))
        filtered = filtered[cols]
        st.dataframe(filtered)

        fig = px.bar(
            data_frame=filtered,
            x='price',
            y='accommodates',
            title=f"{nombre_ciudad}: Alojamiento según precio <= {price_limit}",
            width=700,
            height=400
        )
        st.plotly_chart(fig)

    mostrar_precio_ciudad(df, "Roma")
    mostrar_precio_ciudad(dfM, "Ciudad de México")
    mostrar_precio_ciudad(dfV, "Venecia")

    # --- Gráfico de dispersión ---
    st.header("Relación entre dos variables numéricas (Gráfico de dispersión)")

    numeric_common = list(
        set(df.select_dtypes(['int64', 'float64']).columns)
        & set(dfM.select_dtypes(['int64', 'float64']).columns)
        & set(dfV.select_dtypes(['int64', 'float64']).columns)
    )

    x_var = st.sidebar.selectbox("Variable eje X", numeric_common, key="x_var_disp")
    y_var = st.sidebar.selectbox("Variable eje Y", numeric_common, key="y_var_disp")

    def scatter_por_ciudad(df_ciudad, nombre_ciudad, bandera_path, color):
        st.subheader(f"{nombre_ciudad}")
        st.image(bandera_path, width=100)

        fig = px.scatter(
            data_frame=df_ciudad,
            x=x_var,
            y=y_var,
            title=f"{nombre_ciudad}: {x_var} vs {y_var}",
            color_discrete_sequence=[color],
            width=700,
            height=500
        )
        st.plotly_chart(fig)

    scatter_por_ciudad(df, "Roma", "broma.png", "blue")
    scatter_por_ciudad(dfM, "Ciudad de México", "bmexico.jpg", "green")
    scatter_por_ciudad(dfV, "Venecia", "bvenecia.jpg", "red")


#######################################################################################################################################



#####################################################################
#Viwe 3
#####################################################################
if View == 'Regresion Lineal Simple':
    st.title('Regresión Lineal Simple')

    default_room_type = 'Entire home/apt'
    default_x = 'beds'
    default_y = 'price'

    if "room_type_value" not in st.session_state:
        st.session_state["room_type_value"] = default_room_type
    if "column_1" not in st.session_state:
        st.session_state["column_1"] = default_x
    if "column_2" not in st.session_state:
        st.session_state["column_2"] = default_y

    st.session_state["room_type_value"] = st.sidebar.selectbox(
        "Selecciona un tipo de habitación:",
        df1['room_type'].dropna().unique(),
        index=df1['room_type'].dropna().unique().tolist().index(st.session_state["room_type_value"])
    )

    st.session_state["column_1"] = st.sidebar.selectbox(
        "Selecciona la variable independiente (X):",
        df1.select_dtypes(include=['float', 'int']).columns,
        index=list(df1.select_dtypes(include=['float', 'int']).columns).index(st.session_state["column_1"])
    )

    st.session_state["column_2"] = st.sidebar.selectbox(
        "Selecciona la variable dependiente (Y):",
        df1.select_dtypes(include=['float', 'int']).columns,
        index=list(df1.select_dtypes(include=['float', 'int']).columns).index(st.session_state["column_2"])
    )

    st.write(f"Tipo de habitación seleccionado: {st.session_state['room_type_value']}")
    st.write(f"Variable X: {st.session_state['column_1']}")
    st.write(f"Variable Y: {st.session_state['column_2']}")
    def analizar_ciudad(df, nombre_ciudad, bandera_path, color):
        st.subheader(f"{nombre_ciudad}")
        st.image(bandera_path, width=100)

        room_type_value = st.session_state["room_type_value"]
        column_1 = st.session_state["column_1"]
        column_2 = st.session_state["column_2"]

        df_filtrado = df[df['room_type'] == room_type_value].dropna(subset=[column_1, column_2])

        if df_filtrado.shape[0] < 2:
            st.warning(f"No hay suficientes datos para {nombre_ciudad}.")
            return

        X = df_filtrado[[column_1]]
        y = df_filtrado[column_2]

        modelo = LinearRegression()
        modelo.fit(X, y)
        predicciones = modelo.predict(X)

        r2 = modelo.score(X, y)
        r = np.sqrt(r2)

        st.write(f"Coeficiente de determinación (R²): {r2:.4f}")
        st.write(f"Coeficiente de correlación (R): {r:.4f}")

        st.subheader(f"Gráfico de dispersión - {nombre_ciudad}")
        fig, ax = plt.subplots()
        sns.scatterplot(data=df_filtrado, x=column_1, y=column_2, color=color, label="Datos reales", ax=ax)

        df_plot = pd.DataFrame({
            column_1: X[column_1].values.flatten(),
            'Predicciones': predicciones
        })

        sns.lineplot(data=df_plot, x=column_1, y='Predicciones', color='black', label='Línea de regresión', ax=ax)
        ax.legend()
        st.pyplot(fig)

    with st.expander("Análisis por ciudad"):
        analizar_ciudad(df1, "Roma", "broma.png", "blue")
        analizar_ciudad(df1M, "Ciudad de México", "bmexico.jpg", "green")
        analizar_ciudad(df1V, "Venecia", "bvenecia.jpg", "red")

    st.sidebar.subheader("Análisis de correlación general")
    if st.sidebar.button("Mostrar matrices de correlación por ciudad"):
        st.header("Matrices de correlación por ciudad")

        def mostrar_matriz(df, ciudad, bandera_path):
            st.subheader(f"{ciudad}")
            st.image(bandera_path, width=60)
            df_num = df.select_dtypes(include=['float', 'int'])
            corr = df_num.corr()

            # Mostrar matriz como tabla
            st.dataframe(corr)

            fig, ax = plt.subplots()
            sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", ax=ax)
            ax.set_title(f"Matriz de correlación - {ciudad}")
            st.pyplot(fig)

        mostrar_matriz(df1, "Roma", "broma.png")
        mostrar_matriz(df1V, "Venecia", "bvenecia.jpg")
        mostrar_matriz(df1M, "México", "bmexico.jpg")


###########################################################################################
#Viwe 4
###########################################################################################
elif View == 'Regresion Lineal Multiple':
    st.title('Regresión Lineal Múltiple')

    default_room_type = 'Entire home/apt'
    if "room_type_value" not in st.session_state:
        st.session_state["room_type_value"] = default_room_type

    st.session_state["room_type_value"] = st.sidebar.selectbox(
        "Selecciona un tipo de habitación:",
        df1['room_type'].dropna().unique(),
        index=df1['room_type'].dropna().unique().tolist().index(st.session_state["room_type_value"])
    )

    columnas_numericas = df1.select_dtypes(include=['float', 'int']).columns

    var_dependiente_mult = st.sidebar.selectbox(
        "Selecciona la variable dependiente (Y):",
        ["Selecciona una opción"] + list(columnas_numericas)
    )

    var_indep_mult = st.sidebar.multiselect(
        "Selecciona las variables independientes (X):",
        list(columnas_numericas)
    )

    st.write(f"Tipo de habitación seleccionado: {st.session_state['room_type_value']}")
    st.write(f"Variable Y: {var_dependiente_mult}")
    st.write(f"Variables X: {var_indep_mult}")

    def analizar_ciudad_mult(df, nombre_ciudad, bandera_path, color):
        st.subheader(f"{nombre_ciudad}")
        st.image(bandera_path, width=100)

        room_type_value = st.session_state["room_type_value"]

        df_filtrado = df[df['room_type'] == room_type_value].dropna(subset=[var_dependiente_mult] + var_indep_mult)

        if df_filtrado.shape[0] < 2:
            st.warning(f"No hay suficientes datos para {nombre_ciudad}.")
            return

        X = df_filtrado[var_indep_mult]
        y = df_filtrado[var_dependiente_mult]

        modelo = LinearRegression()
        modelo.fit(X, y)
        predicciones = modelo.predict(X)

        r2 = modelo.score(X, y)
        r = np.sqrt(r2)

        st.write(f"Coeficiente de determinación (R²): {r2:.4f}")
        st.write(f"Coeficiente de correlación (R): {r:.4f}")

        st.subheader(f"Gráfico de comparación - {nombre_ciudad}")
        fig, ax = plt.subplots()
        sns.scatterplot(x=y, y=predicciones, color=color, label="Real vs Predicho", ax=ax)
        ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2, label="Línea ideal")
        ax.set_xlabel("Valor Real")
        ax.set_ylabel("Valor Predicho")
        ax.legend()
        st.pyplot(fig)

    if var_dependiente_mult != "Selecciona una opción" and len(var_indep_mult) >= 2:
        with st.expander("Análisis por ciudad"):
            analizar_ciudad_mult(df1, "Roma", "broma.png", "blue")
            analizar_ciudad_mult(df1M, "Ciudad de México", "bmexico.jpg", "green")
            analizar_ciudad_mult(df1V, "Venecia", "bvenecia.jpg", "red")


###########################################################################################
#Viwe 5
###########################################################################################
elif View == 'Regresion Logistica':
    st.title('Regresión Logística para las tres ciudades')
    columnas_validas = list(
            set([col for col in df2.columns if re.match(r'^[A-Z]_[a-zA-Z_]+$', col)])
            & set([col for col in df2M.columns if re.match(r'^[A-Z]_[a-zA-Z_]+$', col)])
            & set([col for col in df2V.columns if re.match(r'^[A-Z]_[a-zA-Z_]+$', col)])
        )

    st.sidebar.subheader("Selección de variables ")
    var_dependiente_mult = st.sidebar.selectbox(
        "Variable dependiente (Y):",
        ["Selecciona una opción"] + columnas_validas
    )

    var_indep_mult = st.sidebar.multiselect(
        "Variables independientes (X):",
        columnas_validas
    )

    def aplicar_regresion(nombre_ciudad, df_ciudad, bandera_path):
        st.subheader(nombre_ciudad)
        st.image(bandera_path, width=100)

        if var_dependiente_mult != "Selecciona una opción" and len(var_indep_mult) >= 2:
            X = df_ciudad[var_indep_mult]
            y = df_ciudad[var_dependiente_mult]

            if sorted(y.unique()) == [0, 1]:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
                escalar = StandardScaler()
                X_train = escalar.fit_transform(X_train)
                X_test = escalar.transform(X_test)

                modelo = LogisticRegression()
                modelo.fit(X_train, y_train)
                y_pred = modelo.predict(X_test)

                matriz = confusion_matrix(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                exactitud = accuracy_score(y_test, y_pred)
                sensibilidad = recall_score(y_test, y_pred)

                st.markdown("**Matriz de Confusión:**")
                st.table(matriz)
                st.markdown(f"**Precisión:** {round(precision, 4)}")
                st.markdown(f"**Exactitud:** {round(exactitud, 4)}")
                st.markdown(f"**Sensibilidad:** {round(sensibilidad, 4)}")
            else:
                st.error("La variable dependiente debe ser binaria (0 y 1).")
        else:
            st.info("Selecciona una variable dependiente válida y al menos dos independientes.")

    # Mostrar resultados para las 3 ciudades
    aplicar_regresion("Roma", df2, "broma.png")
    aplicar_regresion("Ciudad de México", df2M, "bmexico.jpg")
    aplicar_regresion("Venecia", df2V, "bvenecia.jpg")

Overwriting Proyecto_Final.py
